<a href="https://colab.research.google.com/github/JosPolfliet/vlerick-mai-nlp-2023/blob/main/Assignment%201%20-%20Initial%20data%20prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DAX ESG Dataset

Analysis
https://colab.research.google.com/drive/1TrBuB3BbI5oOskD4ht7aHaAYyporTR7n?usp=sharing
https://www.kaggle.com/datasets/equintel/dax-esg-media-dataset?resource=download
[Detecting greenwashing signals through a comparison of ESG reports and public media](https://www.swisstext.org/wp-content/uploads/2023/09/Greenwashing.pdf )



In [ ]:
from tqdm import tqdm
import torch
tqdm.pandas()
import pandas as pd
import numpy as np

In [ ]:

df = pd.read_csv("https://media.githubusercontent.com/media/JosPolfliet/vlerick-mai-nlp-2023/main/DATA/esg_documents_for_dax_companies.csv", delimiter="|").drop(columns=["Unnamed: 0"])

In [ ]:
df

,company,content,datatype,date,domain,esg_topics,internal,symbol,title,url
0,Beiersdorf AG,Sustainability Highlight Report CARE BEYOND SK...,sustainability_report,2021-03-31,NaN,"['CleanWater', 'GHGEmission', 'ProductLiabilit...",1,BEI,BeiersdorfAG Sustainability Report 2021,NaN
1,Deutsche Telekom AG,Corporate Responsibility Report 2021 2 Content...,sustainability_report,2021-03-31,NaN,"['DataSecurity', 'Iso50001', 'GlobalWarming', ...",1,DTE,DeutscheTelekomAG Sustainability Report 2021,NaN
2,Vonovia SE,VONOVIA SE SUSTAINABILITY REPORT 2021 =For a S...,sustainability_report,2021-03-31,NaN,"['Whistleblowing', 'DataSecurity', 'Vaccine', ...",1,VNA,VonoviaSE Sustainability Report 2021,NaN
3,Merck KGaA,Sustainability Report 2021 TABLE OF CONTENTS S...,sustainability_report,2021-03-31,NaN,"['DataSecurity', 'DataMisuse', 'DrugResistance...",1,MRK,MerckKGaA Sustainability Report 2021,NaN
4,MTU,Our ideas and concepts FOR A SUSTAINABLE FUTUR...,sustainability_report,2020-03-31,NaN,"['WorkLifeBalance', 'Corruption', 'AirQuality'...",1,MTX,MTUAeroEngines Sustainability Report 2020,NaN
...,...,...,...,...,...,...,...,...,...,...
11543,Zalando SE,"To reset your pass, please enter your email be...",tech,2021-07-12,pocketgamer,"['CustomerService', 'Meat', 'HumanCapital', 'T...",0,ZAL,Pocket Gamer Connects Digital # 7,https://www.pocketgamer.biz/events/1636/pocket...
11544,Zalando SE,Lo esencial: Materias primas: correlaciones e...,business,2022-07-05,marketscreener,['WasteManagement'],0,ZAL,"Índices: ni Value ni Growth, ¡defensivos!",https://es.marketscreener.com/noticias/ultimas...
11545,Zalando SE,A look at the evolution of the fashion industr...,thinktank,2021-05-11,cbinsights,"['CustomerService', 'Recruiting', 'Sustainable...",0,ZAL,The Future of Fashion: Technology & the Industry,https://www.cbinsights.com/research/fashion-te...
11546,Zalando SE,"By clicking submit, you agree to share your em...",tech,2022-05-23,techfundingnews,"['ClimateTech', 'Cybersecurity', 'CorporateCul...",0,ZAL,These are the top 10 EMEA VCs of 2022 — TFN,https://techfundingnews.com/these-are-the-top-...


## Add actions to df

In [ ]:
df["content"].iloc[1]

"Desktop Metal's latest acquisition gives it rubber materials and parts that it can vertically integrate with its printers. Desktop Metal said it will acquire Adaptive3D, which makes biocompatible rubber and rubber-like materials for additive manufacturing. The acquisition gives Desktop Metal an entry into volume photopolymer elastomer parts. Adaptive3D's primary resin is Elastic ToughRubber 90 and it 3D prints parts for multiple industries including healthcare, industrial, transportation and oil and gas. Adaptive3D's core technology was developed through Defense Advanced Research Projects Agency ( DARPA) funding. Here's a look at the types of parts Adaptive3D makes. Ric Fulop, CEO of Desktop Metal, said Adaptive3D is part of the additive manufacturing company's vertical integration and proprietary materials plan. Fulop said Desktop Metal will combine Adaptive3D's elastomer materials with its printers. Separately, Desktop Metal reported first quarter revenue of $ 11.3 million, up 35% f

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

tokenizer_name = "ESGBERT/EnvironmentalBERT-action"
model_name = "ESGBERT/EnvironmentalBERT-action"

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

## Slow but readable

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (

In [ ]:
s = df["content"].iloc[1]

MAX_SEQ_LENGTH = 512
OVERLAP = 25
CHUNK_SIZE = MAX_SEQ_LENGTH - OVERLAP

def chunked_predict(s):
    try:
      inputs = tokenizer(str(s), return_tensors="pt")

      total_sequence_length = inputs["input_ids"].shape[1]
      n_chunks = total_sequence_length//CHUNK_SIZE+1

      scores = np.zeros(n_chunks)
      for i in range(0, n_chunks):
          chunk = inputs["input_ids"][0][i*CHUNK_SIZE:i*CHUNK_SIZE+MAX_SEQ_LENGTH].to(device)
          pred = model(chunk.unsqueeze(0))[0]
          scores[i] = pred.softmax(1)[0][1].item() # Predicted confidence score that the text of this chunk is about an action
          # print(f"Chunk {i}: {probs[i]:.3f}", tokenizer.decode(chunk))

      return pd.Series({"mean_action_score": np.mean(scores),"n_chunks": n_chunks, "n_estimated_actions": np.sum(np.array(scores)>0.8)})
    except:
      return pd.Series({"mean_action_score": np.NaN,"n_chunks": np.NaN, "n_estimated_actions": np.NaN})

# print(chunked_predict(s))
# print(chunked_predict("Failure to comply with legal or regulatory requirements and/or changes."))
# print(chunked_predict("Failure to comply with legal or regulatory requirements and/or changes. We reduced water usage by 23% in 2020."))
# df["content"].head(20).progress_apply(chunked_predict)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


nan

In [ ]:

df[["mean_action_score", "n_chunks", "n_estimated_actions"]] = df["content"].progress_apply(chunked_predict)

df.to_pickle("/content/drive/MyDrive/Vlerick - NLP Course/esg_documents_for_dax_companies_with_scores_all.pkl")

100%|██████████| 11548/11548 [12:58<00:00, 14.83it/s]


In [ ]:
df

,company,content,datatype,date,domain,esg_topics,internal,symbol,title,url,wc,mean_score,n_chunks,n_estimated_actions
16,Daimler AG,NaN,annual_report,Store-03-31,NaN,[],1,NaN,unknown Annual Report Store,NaN,1,0.000083,1.0,0.0
92,Adidas AG,Desktop Metal's latest acquisition gives it ru...,tech,2021-05-17,zdnet,['Privacy'],0,ADS,"Desktop Metal acquires Adaptive3D, Q1 revenue ...",https://www.zdnet.com/article/desktop-metal-ac...,573,0.000543,2.0,0.0
93,Adidas AG,Victoria Beckham'sÂ style should never be unde...,general,2022-02-19,whowhatwear,['GenderDiversity'],0,ADS,Victoria Beckham Style: 42 Looks Anyone Can Copy,https://www.whowhatwear.co.uk/victoria-beckham...,1563,0.000742,5.0,0.0
94,Adidas AG,Gino Conti knows a thing or two about shoes. T...,business,2021-05-14,barrons,['GenderDiversity'],0,ADS,Great Escapes: Vibram Master Cobbler Gino Cont...,https://www.barrons.com/articles/great-escapes...,876,0.671056,3.0,2.0
95,Adidas AG,Gear-obsessed editors choose every product we ...,general,2021-03-18,bicycling,"['Inclusion', 'GenderDiversity']",0,ADS,Adidas and Peloton Clothing Collaboration,https://www.bicycling.com/news/a35843583/adida...,425,0.443895,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11538,Zalando SE,Industry’ s most performant security data plat...,business,2023-04-25,marketscreener,"['Cybersecurity', 'Compliance']",0,ZAL,SentinelOne® Launches The Singularity™ Securit...,https://www.marketscreener.com/quote/stock/SEN...,923,0.428084,3.0,1.0
11539,Zalando SE,This Week in Fashion The luxury giant has spen...,business,2021-06-25,businessoffashion,"['Governance', 'HumanCapital']",0,ZAL,LVMH’ s Billion Dollar Retail Bet,https://www.businessoffashion.com/briefings/lu...,1991,0.230980,6.0,1.0
11542,Zalando SE,"MARKET WRAPS Watch For: Eurozone, Germany, Fra...",business,2022-03-01,marketscreener,"['Sanction', 'RussianFederation']",0,ZAL,EMEA Morning Briefing: Europe on Back Foot Aga...,https://www.marketscreener.com/news/latest/EME...,1645,0.000049,5.0,0.0
11544,Zalando SE,Lo esencial: Materias primas: correlaciones e...,business,2022-07-05,marketscreener,['WasteManagement'],0,ZAL,"Índices: ni Value ni Growth, ¡defensivos!",https://es.marketscreener.com/noticias/ultimas...,475,0.000032,3.0,0.0
